# AICC Scenario - Call Center Data Analysis

## Introduction
In today's data-driven world, effective customer service is a critical component of any successful business. Call centers, which handle customer inquiries and issues, generate vast amounts of data. This use case will guide you through analyzing call center data using ThanoSQL, a powerful tool for managing and querying data. We will classify call transcripts into categories using various language models (LLMs), and derive key performance metrics such as average call time, satisfaction score, and resolution rate for each category. This analysis can provide valuable insights into customer interactions, helping to improve service quality and overall customer satisfaction.

## Running the Tutorial in Different Environments
This tutorial can be executed both within ThanoSQL Lab and in a local Python/Jupyter environment. Whether you prefer to work directly within ThanoSQL Lab's integrated environment or set up a local development environment on your machine, the instructions provided will guide you through the necessary steps.

## Dataset
We will be working with the following datasets:
- **Counseling Staff Information Table (agents)**: Contains details about the call center agents.
  - `agent_id`: Unique identifier for each agent.
  - `agent_name`: Name of the agent.
- **Consultation Call Metadata Table (calls)**: Records metadata for each call.
  - `call_id`: Unique identifier for each call.
  - `agent_id`: Identifier linking the call to the agent.
  - `satisfaction_score`: Customer satisfaction score for the call.
  - `call_duration`: Duration of the call.
  - `resolution_rate`: Rate at which the call issue was resolved.
- **Consultation Call Transcription Table (transcript)**: Contains the text of the conversations.
  - `call_id`: Identifier linking the call to the transcription.
  - `conversation`: Text of the conversation during the call.
  - `prompt`: Prompt for LLM generation.

## Goals
1. Classify call transcripts into meaningful categories using LLMs.
2. Calculate and analyze the average call time, satisfaction score, and resolution rate for each category.

## Tokens 
To run the models in this tutorial, you will need the following tokens:
- **OpenAI Token**: Required to access all the OpenAI-related tasks when using OpenAI as an engine. This token enables the use of OpenAI's language models for various natural language processing tasks.
- **Huggingface Token**: Required only to access gated models such as Mistral on the Huggingface platform. Gated models are those that have restricted access due to licensing or usage policies, and a token is necessary to authenticate and use these models. For more information, check this [Huggingface documentation](https://huggingface.co/docs/hub/en/models-gated).
Make sure to have these tokens ready before proceeding with the tutorial to ensure a smooth and uninterrupted workflow.

## Displaying ThanoSQL Query Results in Jupyter Notebooks
The check_result function is designed to handle and display the results of a database query executed via the ThanoSQL client. It ensures that any errors are reported, and successful query results are displayed in a user-friendly format.

**Note: This function is specifically designed to work in Jupyter notebook environments.**

In [17]:
from IPython.display import display

def check_query_result(query_result):
    if query_result.error_result:
        print(query_result.error_result)
    else:
        if query_result.records is not None and len(query_result.records.data) > 0:
            df = query_result.records.to_df()
            display(df)
        else:
            print("Query executed successfully")

## Procedure

### Download Datasets

First, we will download the datasets. 

In [2]:
!wget -O use_case_1_data.zip https://raw.githubusercontent.com/smartmind-team/assets/main/datasets/use_cases/use_case_1_data.zip

--2024-07-04 07:57:54--  https://raw.githubusercontent.com/smartmind-team/assets/main/datasets/use_cases/use_case_1_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9053 (8.8K) [application/zip]
Saving to: ‘use_case_1_data.zip’

use_case_1_data.zip 100%[===================>]   8.84K  --.-KB/s    in 0s      

2024-07-04 07:57:54 (20.2 MB/s) - ‘use_case_1_data.zip’ saved [9053/9053]



In [3]:
!unzip use_case_1_data.zip

Archive:  use_case_1_data.zip
  inflating: agents.csv              
  inflating: calls.csv               
  inflating: transcript.csv          


### Import ThanoSQL Library
Import the ThanoSQL library and create a client instance. This client will be used to interact with the ThanoSQL engine.

**You can find your API Token and Engine URL by following these steps:**

1. Go to your workspace’s settings page.
2. Navigate to the "Developer" tab.
3. Locate and copy your API Token and Engine URL.

In [2]:
from thanosql import ThanoSQL
client = ThanoSQL(api_token="your_api_token", engine_url="engine_url")

### Upload Data to Tables
#### Upload the `agents` table which contains details about the call center agents.

In [24]:
table = client.table.upload('agents', 'agents.csv', if_exists='replace')
table.get_records(limit=10).to_df()

,agent_id,agent_name
0,0,Sarah
1,1,Rachel
2,2,John
3,3,Michael
4,4,David
5,5,Kevin
6,6,Jason
7,7,Jessica
8,8,Ryan
9,9,Matthew


This step uploads the `agents` data to ThanoSQL and retrieves the first 10 records to confirm the upload.

#### Upload the `calls` table which records metadata for each call.

In [25]:
table = client.table.upload('calls', 'calls.csv', if_exists='replace')
table.get_records(limit=10).to_df()

,call_id,agent_id,satisfaction_score,call_duration,resolution_rate
0,1,5,5,8,100
1,2,0,5,10,100
2,3,10,5,14,80
3,4,7,5,9,80
4,5,12,5,14,70
5,6,17,5,20,100
6,7,11,5,12,80
7,8,0,5,8,70
8,9,3,5,15,86
9,10,13,5,12,70


This step uploads the `calls` data to ThanoSQL and retrieves the first 10 records to confirm the upload.

#### Upload the `transcript` table which contains the text of the conversations.

In [26]:
table = client.table.upload('transcript', 'transcript.csv', if_exists='replace')
table.get_records(limit=10).to_df()

,call_id,conversation
0,1,Assistant: Hello! Thank you for reaching out t...
1,2,Assistant: Hello! Thank you for contacting Dig...
2,3,Assistant: Hello! Thank you for reaching out t...
3,4,Assistant: Hello! Thank you for contacting Dig...
4,5,Assistant: Hello! Thank you for contacting Dig...
5,6,Assistant: Hello! Thank you for contacting Dig...
6,7,Assistant: Hello! Thank you for reaching out t...
7,8,Assistant: Hello! Thank you for contacting Dig...
8,9,Assistant: Hello! Thank you for reaching out t...
9,10,Assistant: Hello! Thank you for contacting Dig...


This step uploads the `transcript` data to ThanoSQL and retrieves the first 10 records to confirm the upload.

## Classify Conversations and Aggregate Metrics

#### Classify conversations using the Mistral LLM and calculate performance metrics.

Using Mistral LLM

In [33]:
query_result = client.query.execute("""
    -- This query categorizes call transcripts using a Huggingface model and computes average metrics for each agent and category.

    -- Step 1: Cleanup resources before starting the main query
    SELECT thanosql.cleanup_resources();

    -- Step 2: Categorize call transcripts and compute average metrics
    SELECT 
        t.category,
        COUNT(*),
        AVG(c.satisfaction_score) AS avg_satisfaction_score,  
        AVG(c.call_duration) AS avg_call_duration,            
        AVG(c.resolution_rate) AS avg_resolution_rate         
    FROM 
        (
            -- Generate categories for call transcripts using a Huggingface model
            SELECT 
                t.call_id,                  
                thanosql.generate(
                    input := CONCAT(
                        '<s>[INST]You are a classification assistant. Your task is to analyze a conversation between an assistant and a human and determine the most appropriate category for the conversation <<<>>> into one of the following predefined categories: Technology, Refund, Shipping, ACCOUNT, CANCELLATION_FEE, CONTACT, DELIVERY, Feedback, Invoice, Newsletter, Order\n\nYou will only respond with one of the predefined categories. Reply with capital letters only. Do not start with category or categories. Do not provide explanations or notes. Do not change the line.[/INST]\n\n<<<\nConversation: ',
                        conversation,
                        '</s>\n>>>'
                    ),       
                    engine := 'thanosql', 
                    model := 'smartmind/Mistral-7B-Instruct-v0.2', 
                    model_args := '{"max_new_tokens": 7}' 
                ) AS category              
            FROM
                transcript AS t                
        ) AS t
    JOIN 
        calls AS c ON t.call_id = c.call_id 
    GROUP BY 
        t.category            
""")
check_query_result(query_result)

,category,count,avg_satisfaction_score,avg_call_duration,avg_resolution_rate
0,TECHNOLOGY,27,4.962963,8.222222,89.851852
1,FEEDBACK,6,4.666667,3.000000,94.166667
2,DELIVERY,10,4.700000,3.700000,91.500000
3,NEWSLETTER,3,5.000000,2.333333,100.000000
4,CONTACT,2,4.500000,4.000000,92.500000
5,SHIPPING,9,5.000000,5.555556,100.000000
6,ACCOUNT,35,4.942857,4.742857,95.571429
7,REFUND,6,5.000000,5.500000,99.166667
8,CANCELLATION_,9,5.000000,4.444444,99.555556
9,ORDER,5,5.000000,3.600000,100.000000


#### Classify conversations using the OpenAI GPT-4o and calculate performance metrics.

Using OpenAI GPT-4o

In [34]:
query_result = client.query.execute("""
    -- This query categorizes call transcripts using a model from OpenAI, computes average metrics for each category.
    
    -- Step 1: Cleanup resources before starting the main query
    SELECT thanosql.cleanup_resources();

    -- Step 2: Categorize call transcripts and compute average metrics
    SELECT 
        t.category,
        COUNT(*),              
        AVG(c.satisfaction_score) AS avg_satisfaction_score,   
        AVG(c.call_duration) AS avg_call_duration,             
        AVG(c.resolution_rate) AS avg_resolution_rate          
    FROM 
        (
            -- Step 3: Generate categories for call transcripts using a model from OpenAI
            SELECT 
                t.call_id,                  
                thanosql.generate(
                    input := CONCAT(
                        '<s>[INST]You are a classification assistant. Your task is to analyze a conversation between an assistant and a human and determine the most appropriate category for the conversation <<<>>> into one of the following predefined categories: Technology, Refund, Shipping, ACCOUNT, CANCELLATION_FEE, CONTACT, DELIVERY, Feedback, Invoice, Newsletter, Order\n\nYou will only respond with the category. Do not include the word "Category". Do not provide explanations or notes. Do not change the line.[/INST]\n\n<<<\nConversation: ',
                        conversation,
                        '</s>\n>>>'
                    ),       
                    engine := 'openai',    
                    model := 'gpt-4o',     
                    token := 'your_openai_api_key', 
                    model_args := '{"temperature": 0}' 
                ) AS category             
            FROM
                transcript AS t                
        ) AS t
    JOIN 
        calls AS c ON t.call_id = c.call_id 
    GROUP BY 
        t.category
    ORDER BY
        t.category;
""")
check_query_result(query_result)

,category,count,avg_satisfaction_score,avg_call_duration,avg_resolution_rate
0,ACCOUNT,36,4.944444,4.527778,95.694444
1,CANCELLATION_FEE,7,5.000000,5.000000,99.428571
2,CONTACT,4,4.750000,4.250000,93.750000
3,DELIVERY,11,4.727273,3.545455,92.272727
4,Feedback,5,4.800000,3.000000,95.000000
5,Invoice,9,5.000000,3.777778,98.888889
6,Newsletter,3,5.000000,2.333333,100.000000
7,Order,9,5.000000,3.333333,100.000000
8,Refund,8,5.000000,5.625000,99.375000
9,Shipping,7,5.000000,6.000000,100.000000


## Conclusion
By following these steps, you can effectively analyze call center data, classify call transcripts into meaningful categories using natural language processing, and derive valuable insights into call durations, customer satisfaction, and resolution rates for each category. These insights can help you understand the performance of your call center agents and identify areas for improvement, ultimately leading to enhanced customer satisfaction and operational efficiency.